In [1]:
import nd2
import numpy as np
from skimage.io import imsave,imread
import os
import subprocess
import h5py
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from scipy.optimize import curve_fit
import shutil
from skimage import measure
from matplotlib.path import Path
import seaborn as sns

In [3]:
# nd2_dir = '/Users/zhengj10/Desktop/Test'
nd2_dir = '/Volumes/imaging/Abhi/T-GECO and GCaMP Oct 5 2022'
sum_dir = os.path.join(nd2_dir,'summary')
if not os.path.exists(sum_dir):
    os.makedirs(sum_dir)
    
save_dir = os.path.join(nd2_dir,'summary_221021')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
ilastik_location = '/Applications/ilastik-1.4.0b15-OSX.app/Contents/ilastik-release'
ilastik_project = '/Volumes/imaging/Abhi/ilastik/Ca_sensors.ilp'
os.chdir(ilastik_location)

### Image processing
- Create ref images for ilastik training (needed only once)
- Convert .nd files to .tif
- ROI segmentation with trained ilastik classifier
- Calculate parameters (dFF, kinetics, etc)

In [4]:
stim_onset = {
    '1AP':99-3,
    '3AP':501-3,
    '10AP':907-3,
    '20AP':1326-3,
    '160AP':1768-3
}

In [5]:
coordinates = []
for r in range(1022):
    for c in range(1024):
        coordinates.append([r,c])
coordinates = np.array(coordinates)

In [6]:
f = '/Volumes/imaging/Abhi/FR-GECO and controls FINAL Oct 23 2022/FR-GECO1a_W01.nd2'
with nd2.ND2File(f) as ndfile:
    xarr = ndfile.asarray()
    print(xarr.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/imaging/Abhi/FR-GECO and controls FINAL Oct 23 2022/FR-GECO1a_W01.nd2'

In [ ]:
# sessions = os.listdir(nd2_dir)
# sessions = [s for s in sessions if os.path.isdir(os.path.join(nd2_dir,s))]
# sessions = [s for s in sessions if s!='summary']
sessions = ['GCaMP6s']

# wells = ['W01','W02','W03','W04']
wells = ['W01']

Session = []
Well = []
Stim = []
dFF = []
dFF_peak = []
F0 = []
Fb = []
T_peak = [] # Frame # at dFF trace peak - Frame # at stim onset
Half_rise = [] # Frame # at half dFF trace peak in the rise phase
Half_decay = [] # Frame # at half dFF trace peak in the decay phase
Tau_decay = []
ROI_no = []
ROI_coordinates = []
SNR = []

for s in sessions:
    nd2_files = glob(os.path.join(nd2_dir,s)+'/*.nd2')
    for w in wells:
    
        ref_dir = os.path.join(nd2_dir,s,w,'Ref')
        if not os.path.exists(ref_dir):
            os.makedirs(ref_dir)
            
        tif_dir = os.path.join(nd2_dir,s,w,'Tif')
        if not os.path.exists(tif_dir):
            os.makedirs(tif_dir)
            
        seg_dir = os.path.join(nd2_dir,s,w,'Seg')
        if not os.path.exists(seg_dir):
            os.makedirs(seg_dir)
        
        nd2_w = [f for f in nd2_files if w in os.path.basename(f)]
        for f in nd2_w:
            with nd2.ND2File(f) as ndfile:

                xarr = ndfile.asarray()
                print(xarr.shape)
                mean = np.mean(xarr[10:20,:,:],axis=0)
                print(mean.shape)
                imsave(ref_dir+'/AVG_%s.tif' %os.path.basename(f).split('.')[0],mean)
                imsave(tif_dir+'/%s.tif' %os.path.basename(f).split('.')[0],xarr)
            
        indir = ref_dir
        infiles = os.listdir(indir)
        infiles = [f for f in infiles if 'tif' in f]
        os.chdir(ilastik_location)
        
        for infile in infiles:
            if 'AVG' in infile:

                command = '/Applications/ilastik-1.4.0b15-OSX.app/Contents/ilastik-release/run_ilastik.sh --headless --project="%s" --export_source="simple segmentation" --output_filename_format="%s/%s_ilastik_segmentation.h5" --raw_data="%s/%s"' % (
                            ilastik_project,
                            seg_dir,
                            os.path.basename(infile.split('_')[1]),
                            indir,
                            infile)
                subprocess.run(command, shell=True)

        seg_file = glob(seg_dir+'/*.h5')
        seg_file = seg_file[0]
        seg = h5py.File(seg_file,'r')
        seg = seg['exported_data'][()]
        seg = np.squeeze(seg)
        seg = seg==1
        
        contours = measure.find_contours(seg,0.5)
        
        thre = 75 # Exclude smalll rois
        roi_no = 0
        for contour in contours:
            if contour.shape[0]>thre:
                p = Path(contour)
                roi = p.contains_points(coordinates)
                roi = roi.reshape(1022,1024)
                
#                 print(roi_no)
        
                tif_files = glob(tif_dir+'/*.tif')
                img = tif_files[0]
                img_arr = imread(img)
                c = (seg*roi)*img_arr[3:]
                d = np.sum(c,axis=1)
                F = np.sum(d,axis=1)/np.sum(seg*roi)
                f0 = np.mean(F[10:20])
                fb = 400
        
                L = [0]
                if s != 'T-GECO1':
                    for st in ['1AP','3AP','10AP']:
                        L.append(stim_onset[st]-2)
                else:
                    for st in ['1AP','3AP','10AP','20AP','160AP']:
                        L.append(stim_onset[st]-2)
                x = np.array(L)
                y = F[L]
                try:
                    p,_ = curve_fit(lambda t,a,b,c:a*np.exp(b*t)+c,x,y,p0=(F[0]+F[L][-1],-3.25508359e-03,F[L][-1]))
                    x1 = np.arange(len(F))
                    y1 = p[0]*np.exp(p[1]*x1)+p[2]
                    F_c = F-y1+F[10] # bleach corrected trace
                    dff = (F_c-f0)/(f0-fb)
                except:
                    print('%s %s %s can not be fit with single exp curve' %(s,w,roi_no))
                    p = np.nan
                    dff = (F_c-f0)/(f0-fb)
                
                roi_no+=1

                for st in stim_onset.keys():
                    stim_start_idx = stim_onset[st]
                    if st=='160AP':
                        trace = dff[stim_start_idx:]
                    else:
                        trace = dff[stim_start_idx:stim_start_idx+350]
                    dff_peak = np.max(trace)
                    half_peak = dff_peak/2.0
                    t_peak_idx = np.argmin(np.abs(trace - dff_peak))

                    Session.append(s)
                    Well.append(w)
                    ROI_no.append(roi_no)
                    ROI_coordinates.append(roi)
                    Stim.append(st)
                    if st=='160AP':
                        dFF.append(dff[stim_start_idx-50:])
                    else:
                        dFF.append(dff[stim_start_idx-50:stim_start_idx+350])
                    dFF_peak.append(dff_peak)
                    
                    F0.append(f0)
                    Fb.append(fb)
                    SNR.append(dff_peak/np.std(dff[10:20]))
                    T_peak.append(t_peak_idx)

                    rise_trace = trace[:t_peak_idx]
                    half_rise_idx = np.argmin(np.abs(rise_trace-half_peak))
                    Half_rise.append(half_rise_idx)

                    if st == '160AP':
                        decay_trace = trace[316:]
                    else:
                        decay_trace = trace[t_peak_idx:]
                    half_decay_idx = np.argmin(np.abs(decay_trace-half_peak))
                    Half_decay.append(half_decay_idx)

                np.save(sum_dir+'/%s_%s_Bleach_fit_roi#%s.npy' %(s,w,roi_no),p)
                np.save(sum_dir+'/%s_%s_raw_F_roi#%s.npy' %(s,w,roi_no),F)
                np.save(sum_dir+'/%s_%s_dFF_all_roi#%s.npy' %(s,w,roi_no),dff)

- Calculating dFF with 1AP baseline

### Plot segmentation

In [ ]:
nd2_dir = '/Volumes/imaging/Abhi/T-GECO and GCaMP Oct 5 2022'
# sessions = os.listdir(nd2_dir)
# sessions = [s for s in sessions if os.path.isdir(os.path.join(nd2_dir,s))]
# sessions = [s for s in sessions if s!='summary']
sessions = ['T-GECO1','GCaMP8s','GCaMP6s']
wells = ['W03']

In [ ]:
coordinates = []
for r in range(1022):
    for c in range(1024):
        coordinates.append([r,c])
coordinates = np.array(coordinates)

In [ ]:
for s in sessions:
    plt.figure(figsize=[10,5])
    for w in wells:
        ref_dir = os.path.join(nd2_dir,s,w,'Ref')
        ref_img = glob(ref_dir+'/*.tif')
        ref_img = ref_img[0]
        ref_img = imread(ref_img)

        ax1 = plt.subplot(121)
        vmax = np.median(ref_img)*1.5
        ax1.imshow(ref_img,cmap='gray',vmax=vmax)
        ax1.axis('off')
        ax1.text(30,40,s+' '+w,color='white',weight='bold')
    
        seg_dir = os.path.join(nd2_dir,s,w,'Seg')
        seg_file = glob(seg_dir+'/*.h5')
        seg_file = seg_file[0]
        seg = h5py.File(seg_file,'r')
        seg = seg['exported_data'][()]
        seg = np.squeeze(seg)
        seg = seg==1
        
        ax2 = plt.subplot(122)
        ax2.imshow(seg,cmap='gray')
        ax2.axis('off')
#         ax2.set_title(s+' '+w)
        ax2.text(30,40,s+' '+w,color='white',weight='bold')
        thre = 75 # Exclude smalll rois
        rois = []
        contours = measure.find_contours(seg,0.5)
        for contour in contours:
            if contour.shape[0]>thre:
                ax2.plot(contour[:,1],contour[:,0],color='lime',lw=1.5)
                p = Path(contour)
                roi = p.contains_points(coordinates)
                roi = roi.reshape(1022,1024)
                rois.append(roi)
                
    plt.tight_layout(pad=0.25)
    sum_dir = os.path.join(nd2_dir,'summary')
    if not os.path.exists(sum_dir):
        os.makedirs(sum_dir)
#     plt.savefig(sum_dir+'/%s_img_seg.png' %s,dpi=1200)
    plt.show()
    plt.close()

#### Last ROI in above loop to confirm the location matches seg file

In [ ]:
plt.figure(figsize=[10,5])
ax1 = plt.subplot(121)
# ax1.imshow(seg.astype(np.float32)*roi.astype(np.float32),cmap='gray')
ax1.imshow(seg*rois[2],cmap='gray')
ax1.axis('off')
ax2 = plt.subplot(122)
ax2.imshow(seg,cmap='gray')
ax2.axis('off')
plt.tight_layout(pad=0.25)
plt.show()

### Save the data
- pkl file for reading and plotting in notebook
- excel file for reading and plotting in other programs

In [ ]:
df_new = pd.DataFrame()
df_new['Session'] = Session
df_new['Well'] = Well
df_new['ROI_#'] = ROI_no
df_new['ROI_coordinates'] = ROI_coordinates
df_new['Stim'] = Stim
df_new['dFF'] = dFF
df_new['dFF_peak'] = dFF_peak
df_new['F0'] = F0
df_new['Fb'] = Fb
df_new['SNR'] = SNR
df_new['T_peak'] = T_peak
df_new['Half_rise'] = Half_rise
df_new['Half_decay'] = Half_decay

In [ ]:
# df = df_new # For the first experiment

In [3]:
# df = pd.DataFrame() # Create empty dataframe for first experiment
df = pd.read_pickle(sum_dir+'/Ca_sensor_screen.pkl')
# df = pd.concat([df,df_new],axis=0) # Append new data to existing dataframe
# df

In [12]:
# df_n = df
df.Stim.unique()

array(['1AP', '3AP', '10AP', '20AP', '160AP'], dtype=object)

In [ ]:
df_n.to_pickle(save_dir+'/Ca_sensor_screen.pkl')
df_n.to_excel(save_dir+'/Ca_sensor_screen.xlsx')

In [19]:
df_dFF = pd.DataFrame()

In [20]:
df_dFF['Time (s)'] = np.arange(1,df.loc[0,'dFF'].shape[0]+1)/33.3
for session in df.Session.unique():
    for stim in df.Stim.unique():
    
        df_temp = df[(df['Stim']==stim)&(df['Session']==session)]
        for i,r in df_temp.iterrows():
            s = pd.Series(r['dFF'])
            df_dFF = pd.concat((df_dFF,s.rename(r['Session']+' '+r['Stim']+' '+r['Well']+' ROI#_'+str(r['ROI_#']))),axis=1)

In [21]:
df_dFF

,Time (s),GCaMP6s 1AP W01 ROI#_1,GCaMP6s 1AP W01 ROI#_2,GCaMP6s 1AP W01 ROI#_3,GCaMP6s 1AP W01 ROI#_4,GCaMP6s 1AP W01 ROI#_5,GCaMP6s 1AP W01 ROI#_6,GCaMP6s 1AP W01 ROI#_7,GCaMP6s 1AP W01 ROI#_8,GCaMP6s 1AP W01 ROI#_9,...,T-GECO1 160AP W04 ROI#_51,T-GECO1 160AP W04 ROI#_52,T-GECO1 160AP W04 ROI#_53,T-GECO1 160AP W04 ROI#_54,T-GECO1 160AP W04 ROI#_55,T-GECO1 160AP W04 ROI#_56,T-GECO1 160AP W04 ROI#_57,T-GECO1 160AP W04 ROI#_58,T-GECO1 160AP W04 ROI#_59,T-GECO1 160AP W04 ROI#_60
0,0.030030,0.098612,0.003171,0.204565,0.019430,0.000238,-0.014665,-0.005237,-0.147634,0.027793,...,-0.011855,0.000953,-0.004294,-0.011806,0.211373,-0.118351,-0.003255,1.358739,-0.003974,0.002877
1,0.060060,0.080853,0.003938,0.205486,0.018916,0.022552,-0.000845,-0.010429,-0.152083,0.031865,...,-0.002297,0.021083,-0.003966,-0.008774,0.215091,-0.115646,-0.013627,1.334195,-0.004420,0.003306
2,0.090090,0.071940,-0.018642,0.178373,0.019049,0.004702,-0.010232,-0.004673,-0.147151,0.023337,...,-0.009206,-0.000689,0.002188,-0.011200,0.212116,-0.117810,-0.008109,1.347253,-0.002828,-0.005456
3,0.120120,0.075917,0.002983,0.204339,0.018006,0.027100,-0.013526,-0.008599,-0.150514,0.015622,...,-0.003808,-0.024158,-0.002055,-0.012226,0.210859,-0.118726,-0.003753,1.357559,0.003924,-0.008143
4,0.150150,0.083877,0.006113,0.208098,0.009093,0.007553,-0.000026,0.007294,-0.136896,0.021091,...,0.000311,-0.003438,-0.006336,-0.012747,0.210220,-0.119191,-0.004007,1.356960,-0.006143,-0.008704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,11.891892,0.002741,0.006665,0.208761,-0.000787,0.035906,-0.014272,0.000474,-0.142740,0.028651,...,0.115181,0.244267,0.067328,0.037770,0.272146,-0.074120,0.070174,1.532504,0.142150,0.478423
396,11.921922,0.008507,0.003683,0.205180,-0.009319,0.018870,-0.013688,0.001820,-0.141587,0.019805,...,0.118823,0.246467,0.068244,0.046024,0.282264,-0.066756,0.065798,1.522150,0.127938,0.466281
397,11.951952,-0.018501,0.004103,0.205685,-0.005560,0.031957,-0.016020,0.005294,-0.138610,0.019081,...,0.107189,0.245715,0.063682,0.045018,0.281031,-0.067654,0.065261,1.520878,0.133748,0.473762
398,11.981982,-0.012634,-0.001053,0.199493,0.005047,0.031438,0.002464,-0.011253,-0.152789,0.024320,...,0.126518,0.238841,0.061213,0.042601,0.278068,-0.069810,0.057862,1.503368,0.133973,0.455951


In [22]:
from itertools import zip_longest
df_dFF.columns = pd.MultiIndex.from_arrays(zip_longest(*df_dFF.columns.map(str.split), fillvalue=''))
df_dFF

Time   GCaMP6s                                                    \
           (s)       1AP                                                     
                     W01                                                     
                  ROI#_1    ROI#_2    ROI#_3    ROI#_4    ROI#_5    ROI#_6   
0     0.030030  0.098612  0.003171  0.204565  0.019430  0.000238 -0.014665   
1     0.060060  0.080853  0.003938  0.205486  0.018916  0.022552 -0.000845   
2     0.090090  0.071940 -0.018642  0.178373  0.019049  0.004702 -0.010232   
3     0.120120  0.075917  0.002983  0.204339  0.018006  0.027100 -0.013526   
4     0.150150  0.083877  0.006113  0.208098  0.009093  0.007553 -0.000026   
..         ...       ...       ...       ...       ...       ...       ...   
395  11.891892  0.002741  0.006665  0.208761 -0.000787  0.035906 -0.014272   
396  11.921922  0.008507  0.003683  0.205180 -0.009319  0.018870 -0.013688   
397  11.951952 -0.018501  0.004103  0.205685 -0.005560  0.031957 -0.016020   
398  11.981982 -0.012634 -0.001053  0.199493  0.005047  0.031438  0.002464   
399  12.012012 -0.012887  0.000848  0.201776  0.001421  0.020342 -0.023279   

                                   ...   T-GECO1                      \
                                   ...     160AP                       
                                   ...       W04                       
       ROI#_7    ROI#_8    ROI#_9  ...   ROI#_51   ROI#_52   ROI#_53   
0   -0.005237 -0.147634  0.027793  ... -0.011855  0.000953 -0.004294   
1   -0.010429 -0.152083  0.031865  ... -0.002297  0.021083 -0.003966   
2   -0.004673 -0.147151  0.023337  ... -0.009206 -0.000689  0.002188   
3   -0.008599 -0.150514  0.015622  ... -0.003808 -0.024158 -0.002055   
4    0.007294 -0.136896  0.021091  ...  0.000311 -0.003438 -0.006336   
..        ...       ...       ...  ...       ...       ...       ...   
395  0.000474 -0.142740  0.028651  ...  0.115181  0.244267  0.067328   
396  0.001820 -0.141587  0.019805  ...  0.118823  0.246467  0.068244   
397  0.005294 -0.138610  0.019081  ...  0.107189  0.245715  0.063682   
398 -0.011253 -0.152789  0.024320  ...  0.126518  0.238841  0.061213   
399 -0.008741 -0.150637  0.022573  ...  0.139065  0.266196  0.062420   

                                                                           
                                                                           
                                                                           
      ROI#_54   ROI#_55   ROI#_56   ROI#_57   ROI#_58   ROI#_59   ROI#_60  
0   -0.011806  0.211373 -0.118351 -0.003255  1.358739 -0.003974  0.002877  
1   -0.008774  0.215091 -0.115646 -0.013627  1.334195 -0.004420  0.003306  
2   -0.011200  0.212116 -0.117810 -0.008109  1.347253 -0.002828 -0.005456  
3   -0.012226  0.210859 -0.118726 -0.003753  1.357559  0.003924 -0.008143  
4   -0.012747  0.210220 -0.119191 -0.004007  1.356960 -0.006143 -0.008704  
..        ...       ...       ...       ...       ...       ...       ...  
395  0.037770  0.272146 -0.074120  0.070174  1.532504  0.142150  0.478423  
396  0.046024  0.282264 -0.066756  0.065798  1.522150  0.127938  0.466281  
397  0.045018  0.281031 -0.067654  0.065261  1.520878  0.133748  0.473762  
398  0.042601  0.278068 -0.069810  0.057862  1.503368  0.133973  0.455951  
399  0.037568  0.271898 -0.074300  0.059120  1.506345  0.128614  0.460953  

[400 rows x 2156 columns]

In [23]:
df_dFF.to_excel('/Users/zhengj10/Desktop/Ca_sensor_screen_dFF_only.xlsx')

In [ ]:
plt.rc('font',size=13)
sns.set_style('whitegrid')
# Y_lim={}
fig = plt.figure(figsize=[13,8])
sb = 1
for s in df_n.Session.unique():
    
    for stim in df_n.Stim.unique():

        df_p = df_n[(df_n['Session']==s)&(df_n['Stim']==stim)]
        mean = np.nanmean(df_p.dFF.to_list(),axis=0)
        sem = np.nanstd(df_p.dFF.to_list(),axis=0)/np.sqrt(len(df_p.dFF.to_list()))
        x = np.arange(mean.shape[0])/33.0
        ax = plt.subplot(3,5,sb)
    #     ax = plt.subplot(111)
        ax.plot(x,mean,color='k',alpha=0.75)
        ax.fill_between(x,mean+sem,mean-sem,color='k',alpha=0.25)
#         if stim=='160AP':
#             Y_lim[s] = ax.get_ylim()
#         ax.set_ylim(Y_lim[s])
        if sb in(1,6,11):
            ax.set_ylabel('\u0394F/F0',size=14)
        if sb>10:
            ax.set_xlabel('s',size=14)
#         ax.yaxis.set_label_coords(-.1, .5)
        sb+=1
#     fig.align_ylabels(ax[:, 1])
plt.tight_layout(pad=0.75)
plt.savefig('/Users/zhengj10/Desktop/221025/Abhi_data/dFF.png',dpi=1200)
plt.show()

In [ ]:
Y_lim

In [ ]:
plt.figure(figsize=[13,3.5])
plt.rc('font',size=13)
sb = 1
for s in df_n.Stim.unique():
    df_p = df_n[(df_n['Stim']==s)]
#     df_p = df_p.assign(Half_rise_time = df_p['Half rise time (ms)']/1000)
#     df_p = df_p.assign(Half_decay_time = df_p['Half decay time (ms)']/1000)
    ax = plt.subplot(1,5,sb)
#     sns.stripplot(x='Session',y='Half_rise_time',data=df_p,ax=ax)
#     sns.boxplot(x='Session',y='Half_rise_time',data=df_p,ax=ax)
    sns.stripplot(x='Session',y='Half decay time (ms)',data=df_p,ax=ax)
    sns.boxplot(x='Session',y='Half decay time (ms)',data=df_p,ax=ax)
    if sb==1:
#         ax.set_ylabel('Half rise time (s)',size=14)
        ax.set_ylabel('Half decay time (s)',size=14)
    else:
        ax.set_ylabel('')
    ax.set_xlabel('',size=14)
    sb+=1
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
plt.tight_layout()
# plt.savefig('/Users/zhengj10/Desktop/221025/Abhi_data/hr.png',dpi=1200)
# plt.savefig('/Users/zhengj10/Desktop/221025/Abhi_data/hd.png',dpi=1200)
plt.show()

In [ ]:
c = df_ps[df_ps['Half rise time (ms)']>8000]

In [ ]:
plt.plot(c.dFF.to_list()[0])

In [ ]:
# ax.get_ylim()
df_n.columns

### Delete .tif files

In [ ]:
# for s in sessions:
#     for w in wells:
#         tif_dir = os.path.join(nd2_dir,s,w,'Tif')
#         shutil.rmtree(tif_dir)